In [2]:
import pandas as pd
import numpy as np

# input data 부분 필요 라이브러리
import kqc_custom
from dataclasses import dataclass, field, asdict, astuple
from typing import List, ClassVar
import ast  # 입력 문자 beta coef, integer 로 변환에 사용하는 부분

# Simulated Annealing 필요 라이브러리
from sa_basefunctions import get_aic, get_bic, get_mspe, get_bin, flip
from sa_SimulatedAnnealing import SimulatedAnnealing

# Genetic Algorithm 필요 라이브러리 

## Data Generator 구현 부분

*Data Generating Function 함수 설명*

$y = X \beta + \epsilon$

kqc_custom.generate_independent_sample(n_sample,n_features,beta_coef=[4,3,2,2],epsilon=4)  
kqc_custom.generate_dependent_sample(n_sample,n_features,beta_coef=[4,3,2,2],epsilon=4,correlation_parameter=1)

- n_samples : 데이터 샘플 수
- n_features : 데이터 피쳐 개수
- beta_coef : beta계수
- epsilon : 오차의 표준편차 정도
- correlation_parameter


In [3]:
# inout parameter 가져오기
KQC_data_parameter = pd.read_excel('input_parameter.xlsx')

Example of input data generator funvtion's parameter input from EXCEL (.xlcs file)

    아래에 간단한 예시로 4개의 데이터의 경우를 예시로 들었습니다.

In [4]:
KQC_data_parameter

,지정 데이터명,n_samples,n_features,beta_coef,epsilon,correlation,is_independent
0,data1,500,10,"[4,3,2,2]",1,1,independent
1,data2,500,10,"[4,3,2,2]",2,1,independent
2,data3,500,10,"[4,3,2,2]",3,1,independent
3,data4,500,10,"[4,3,2,2]",4,1,independent


In [5]:
def read_excel2tuple():
    '''
        row 별로 data를 슬라이싱 해주는 Tuple로 변환
    '''
    df = pd.read_excel('input_parameter.xlsx')
    return list(df.itertuples(name=None))

In [6]:
Tupled_df_KQC_data_parameter = list(KQC_data_parameter.itertuples(name=None))

In [7]:
list(list(KQC_data_parameter.itertuples(name=None))[0])

[0, 'data1', 500, 10, '[4,3,2,2]', 1, 1, 'independent']

EXCEL 의 dataframee 의 pandas로 받아서, 

In [8]:
from typing import ClassVar
@dataclass
class data_generator:
    count_data : ClassVar[int] = 1  # 전치 처리위한 class variable

    # KQC custom file // data generator function's parameter
    n_samples : int                 # 데이터 샘플 수
    n_features : int                # 데이터 피쳐 개수
    beta_coef : float               # beta 계수
    epsilon : float                 # 오차의 표준편차 정도
    correlation_parameter : float   # 상관계수 ?

    # kqc data generator dependent / inpependent 선택 부분 
    is_independent : str            # is_independent
    data_name : str                 # 지정 데이터명

    def __post_init__(self):
        data_generator.count_data += 1  # class 생성 data counter
      
    # parameter 입력 받기 
    @classmethod
    def create(cls): 
        data = cls() 
        return data
    
    @classmethod
    def data_counter(cls):
        return [ 'data' + str(i) for i in range(cls.count_data)]

    #KQC_custom 의 data generator 로 부터 data 생성 class
    def data_gen(self):
        data_gened = []
        if self.is_independent == 'dependent':
            x, y = kqc_custom.generate_dependent_sample( self.n_samples,
                                                  self.n_features,
                                                  self.beta_coef,
                                                  self.epsilon,
                                                  self.correlation_parameter
            )
        else:
            x, y = kqc_custom.generate_independent_sample( self.n_samples,
                                                  self.n_features,
                                                  self.beta_coef,
                                                  self.epsilon,
                                                  self.correlation_parameter
            )
        data_gened.append(x)
        data_gened.append(y)
        return data_gened
# data_name : str 
# paeameter : List[float] = field(default_factory=list)

In [9]:
ast.literal_eval(list(list(KQC_data_parameter.itertuples(name=None))[0])[4])

[4, 3, 2, 2]

In [10]:
# class data generator 로 부터, 입력 데이터 리스트의 수 만큼 리스트 생성 
def list_gen_from_class(df):
    generated_data_list = []
    Tupled_df_KQC_data_parameter = read_excel2tuple()
    for i in range(len(Tupled_df_KQC_data_parameter)):
        data_list_Row = list(list(df.itertuples(name=None))[i])
        generated_data_list.append(data_generator(n_samples= data_list_Row[2],
                                                n_features= data_list_Row[3],
                                                beta_coef= ast.literal_eval(data_list_Row[4]),
                                                epsilon= data_list_Row[5],
                                                correlation_parameter= data_list_Row[6],
                                                is_independent= data_list_Row[7],
                                                data_name=data_list_Row[1]
            )
        )
    return generated_data_list

다음은 generated data list 의 형태를 보여주는 예시입니다.  

아래의 예시와 같이 각 리스트에서 KQC_ independent / dependent data 의 생성되는  
x , y 의 값이 N(excel input 데이터수) by 2 의 size 리스트 입니다.  

In [11]:
generated_data_list = list_gen_from_class(KQC_data_parameter)
#generated_data 의 length --> 클래스 변수에서 선언횟수 카운터로 선언했으므로 
# 임의의 첫번쨰 선언 instance 에서 클래스 매서드를 호출하면 조정없이 가능
data_key = generated_data_list[0].data_counter()

리스트와 동일하지만 검색에 유용하기 위해서, 데이터 명으로 검색을 하기 위해서 딕셔너리로 변환하는  부분을 작성해 보았습니다.  

In [12]:
generated_data_dict = {}
for i in range(len(generated_data_list)):
    '''
        클래스에서 선언 인스턴스 개수로 자동 카운트 되는 이름명은 윗 코드
        dataframe 에서 선언한 '지정 데이터명'을 사용한다면 아래 코드를 사용하면 됩니다.
    '''
    generated_data_dict[data_key[i]] = generated_data_list[i]
    # list(KQC_data_parameter.loc[:,'지정 데이터명'])[i] = generated_data_list[i]

In [13]:
generated_data_dict['data3']

data_generator(n_samples=500, n_features=10, beta_coef=[4, 3, 2, 2], epsilon=4, correlation_parameter=1, is_independent='independent', data_name='data4')

In [14]:
# 클래스 구조를 가짐으로서 data 이름을 검색한것으로 부터 parameter를 쉽게 확인할 수 있습니다.
generated_data_dict['data3'].__dict__

{'n_samples': 500,
 'n_features': 10,
 'beta_coef': [4, 3, 2, 2],
 'epsilon': 4,
 'correlation_parameter': 1,
 'is_independent': 'independent',
 'data_name': 'data4'}

In [15]:
# 저장된 클래스 인스턴스로 부터 데이터 생성하기 
generated_data_dict['data3'].data_gen()

[array([[ 1.62434536, -0.61175641, -0.52817175, ..., -0.7612069 ,
          0.3190391 , -0.24937038],
        [ 1.46210794, -2.06014071, -0.3224172 , ..., -0.87785842,
          0.04221375,  0.58281521],
        [-1.10061918,  1.14472371,  0.90159072, ..., -0.93576943,
         -0.26788808,  0.53035547],
        ...,
        [-2.01516818, -0.99690494, -1.03425079, ..., -1.16362557,
          0.06157423, -0.57646002],
        [-1.5929365 ,  0.71568559,  0.9248516 , ..., -0.38246272,
          0.97307106,  1.06971394],
        [-0.54441474, -0.06307104,  0.89381886, ..., -0.77598779,
          1.08488869,  2.24198946]]),
 array([-2.23918967e+00,  2.77062608e+00, -2.67530053e+00, -9.92061076e+00,
         9.54648348e-01,  2.91207576e+00,  2.26725404e+00, -7.23531689e+00,
         4.12832328e+00,  6.09632648e+00,  5.34863852e+00, -1.07366807e+01,
         4.52824111e+00, -7.11761308e+00, -4.09005398e+00, -3.29252435e+00,
         2.54994490e+00,  7.94742650e+00, -3.94756430e-01,  1.2017236

## 알고리즘 적용 부분

### QAOA

In [39]:
from qiskit import Aer,IBMQ
from qiskit.utils import algorithm_globals, QuantumInstance

In [40]:
from QAOA_algorithm import QAOA, result_container
from QAOA_results import qaoa_Results

In [19]:
# 인턴 최동수 토큰 
TOKEN = "3fe453517228f7d730aa0d601175feba50b9e15ad101299a1361277d94a5e3f083c99e1b9d43c27049117b5f42717593b4c9bbd330bf657da722e1ddb35aca4e"
provider = IBMQ.save_account(TOKEN, overwrite = True)
provider = IBMQ.load_account()

In [20]:
provider = IBMQ.load_account()
provider_real = IBMQ.get_provider(hub='ibm-q-kqc', group='kqc-summer-inter', project='base')

ibmqfactory.load_account:WARNING:2022-08-09 17:12:00,142: Credentials are already in use. The existing account in the session will be replaced.


In [23]:
algorithm_globals.massive = True

백엔드 선택 부분 

In [24]:
# 원본 QAOA 함수에서 backend 실행을 위해 분리해줌
# Qasm 시뮬레이터/Real_device 사용시
#backend = provider_real.get_backend("ibm_hanoi")

# 로컬 시뮬레이터 사용시
backend =Aer.get_backend('qasm_simulator')

SA 와 별개로 ga는 실험 데이터별로 사용할 매개변수의 조정이 별도로 없어서 4가지 종류의 input data로 부터
바로 GA 알고리즘을 실행합니다.  
아래는 생성한 data 4개의 딕셔너리 입니다.

In [41]:
qaoa_resutls_dict = {}

for data_name, data in generated_data_dict.items():
    x, y = pd.DataFrame(data.data_gen()[0]), data.data_gen()[1]
    best_solution, result_data = QAOA(x, y, backend)
    new_name = data_name + '_with_' + 'QAOA'
    qaoa_resutls_dict[new_name] = qaoa_Results(best_solution, result_data)

### 최종결과 확인 

#### QAOA

비고  
4개의 데이터를 input_paramer.xlcs 에서 생성해서 데이터를 만듬  
EXCEl 에서 수정을 통해 원하는데이터 개수만큼 생성 가능 

In [44]:
'''
    ga_results_dict 라는 dictionary 에 input 데이터만큼 저장이 됩니다.
    딕셔너리의 키 인 데이터 이름을 추출해 줍니다. 
'''
qaoa_result_keys = list(qaoa_resutls_dict.keys())
qaoa_result_keys

['data0_with_QAOA', 'data1_with_QAOA', 'data2_with_QAOA', 'data3_with_QAOA']

In [45]:
# EXAMPLE
print(qaoa_result_keys[1])
qaoa_reuslt_data_example = qaoa_resutls_dict[qaoa_result_keys[1]]
#이제 데이터의 3가지 값을 아래와 같이 . 변수명 으로 접근해서 실험값을 추출할 수 있습니다.

data1_with_QAOA


##### 결과 출력 예시

In [46]:
qaoa_reuslt_data_example.best_solution

[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]

In [51]:
qaoa_reuslt_data_example.result_log

[result_container(SSE=[0.3722836447914497, 0.21785471331293582, 0.18383684287387314, 0.1186038780384725, 0.005806018957529302, 0.0005424162894756002, 0.001385821607808646, 1.7119895396100482e-05, 0.0006434805415642675, 0.0006727812426415847], AIC=0    3.921127
 dtype: float64, BIC=0    643.043479
 dtype: float64, lst=0    689.404168
 dtype: float64),
 result_container(SSE=Index(['x2', 'x6', 'x8', 'x9'], dtype='object'), AIC=0    27.878726
 dtype: float64, BIC=0    1611.785861
 dtype: float64, lst=0    1632.858902
 dtype: float64),
 result_container(SSE=Index(['x0', 'x1', 'x3', 'x4', 'x5', 'x7'], dtype='object'), AIC=0    8.000243
 dtype: float64, BIC=0    991.58985
 dtype: float64, lst=0    1021.092107
 dtype: float64)]

In [50]:
# 클래스 인스턴스로 출력했기 때문에 QAOA_algorithm.py 에 있는 클래스 method 출력함수 이용가능
qaoa_reuslt_data_example.result_log[0].print_result()

0    689.404168
dtype: float64
SSE : [0.3722836447914497, 0.21785471331293582, 0.18383684287387314, 0.1186038780384725, 0.005806018957529302, 0.0005424162894756002, 0.001385821607808646, 1.7119895396100482e-05, 0.0006434805415642675, 0.0006727812426415847]
AIC : 0    3.921127
dtype: float64
BIC : 0    643.043479
dtype: float64


##### 결과출력 예시

In [53]:
Tupled_df_KQC_data_parameter = list(KQC_data_parameter.itertuples(name=None))

In [58]:
for i in range(len(Tupled_df_KQC_data_parameter)):
    print('data'+str(i+1) + ' : ')
    print('   '+'DATA 의 parameter')
    print('      ' + str(Tupled_df_KQC_data_parameter[i]))
    print('   '+'QAOA의 Bestsolution')
    print('      ' + str(qaoa_resutls_dict[qaoa_result_keys[i]].best_solution))

data1 : 
   DATA 의 parameter
      (0, 'data1', 500, 10, '[4,3,2,2]', 1, 1, 'independent')
   QAOA의 Bestsolution
      [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
data2 : 
   DATA 의 parameter
      (1, 'data2', 500, 10, '[4,3,2,2]', 2, 1, 'independent')
   QAOA의 Bestsolution
      [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
data3 : 
   DATA 의 parameter
      (2, 'data3', 500, 10, '[4,3,2,2]', 3, 1, 'independent')
   QAOA의 Bestsolution
      [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
data4 : 
   DATA 의 parameter
      (3, 'data4', 500, 10, '[4,3,2,2]', 4, 1, 'independent')
   QAOA의 Bestsolution
      [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
